In [1]:
import keras
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D,BatchNormalization
from keras.layers import Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras.datasets import cifar10
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [2]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), 
               padding = 'valid', 
               name = conv_name_base + '2a', 
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path 
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), 
               padding = "same",
               name = conv_name_base + "2b", 
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation("relu")(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), 
               padding = "valid",
               name = conv_name_base + "2c", 
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2c")(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    
    return X

In [3]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers,depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path 
    X = Conv2D(F2, (f, f), strides = (1,1), padding = "same",name = conv_name_base + '2b',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1), strides = (1,1), name = conv_name_base + '2c',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', 
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    
    return X

In [4]:
def ResNet50(input_shape = (32, 32, 3), classes = 10):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK 
    -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 
    -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', 
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], 
                            stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], 
                            stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], 
                            stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], 
                            stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL. Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(1, 1))(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), 
              kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [5]:
model = ResNet50(input_shape = (32, 32, 3), classes = 10)

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train/255
x_test = x_test/255
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

In [25]:
model.fit(x_train, y_train, epochs = 10, batch_size = 32,initial_epoch=1)

Epoch 1/1
50000/50000 [==============================] - 227s 5ms/step - loss: 2.4691 - acc: 0.2596


In [26]:
model.fit(x_train, y_train, epochs = 10, batch_size = 64,initial_epoch=1)

Epoch 2/10
50000/50000 [==============================] - 122s 2ms/step - loss: 2.1882 - acc: 0.2784
Epoch 3/10
50000/50000 [==============================] - 122s 2ms/step - loss: 2.1704 - acc: 0.2877
Epoch 4/10
50000/50000 [==============================] - 122s 2ms/step - loss: 2.0273 - acc: 0.3304
Epoch 5/10
50000/50000 [==============================] - 122s 2ms/step - loss: 2.0520 - acc: 0.3195
Epoch 6/10
50000/50000 [==============================] - 122s 2ms/step - loss: 2.0064 - acc: 0.3328
Epoch 7/10
50000/50000 [==============================] - 122s 2ms/step - loss: 1.8725 - acc: 0.3782
Epoch 8/10
50000/50000 [==============================] - 122s 2ms/step - loss: 1.8910 - acc: 0.3655
Epoch 9/10
50000/50000 [==============================] - 121s 2ms/step - loss: 1.8032 - acc: 0.3936
Epoch 10/10
50000/50000 [==============================] - 121s 2ms/step - loss: 1.6985 - acc: 0.4310


In [27]:
preds = model.evaluate(x_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

10000/10000 [==============================] - 10s 973us/step
Loss = 2.93044428635
Test Accuracy = 0.1968


In [28]:
model.fit(x_train, y_train, epochs = 50, batch_size = 64,initial_epoch=10)

Epoch 11/50
50000/50000 [==============================] - 121s 2ms/step - loss: 1.6595 - acc: 0.4487
Epoch 12/50
50000/50000 [==============================] - 121s 2ms/step - loss: 1.9149 - acc: 0.3698
Epoch 13/50
50000/50000 [==============================] - 121s 2ms/step - loss: 1.7208 - acc: 0.4355
Epoch 14/50
50000/50000 [==============================] - 121s 2ms/step - loss: 1.9220 - acc: 0.3665
Epoch 15/50
50000/50000 [==============================] - 121s 2ms/step - loss: 2.1953 - acc: 0.2561
Epoch 16/50
50000/50000 [==============================] - 121s 2ms/step - loss: 1.7685 - acc: 0.3512
Epoch 17/50
50000/50000 [==============================] - 121s 2ms/step - loss: 1.5967 - acc: 0.4151
Epoch 18/50
50000/50000 [==============================] - 121s 2ms/step - loss: 1.5225 - acc: 0.4451
Epoch 19/50
50000/50000 [==============================] - 121s 2ms/step - loss: 1.4357 - acc: 0.4760
Epoch 20/50
50000/50000 [==============================] - 121s 2ms/step - loss: 1

In [29]:
preds = model.evaluate(x_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

10000/10000 [==============================] - 9s 866us/step
Loss = 4.51188512497
Test Accuracy = 0.3336


In [30]:
model.fit(x_train, y_train, epochs = 60, batch_size = 64,initial_epoch=50)

Epoch 51/60
50000/50000 [==============================] - 121s 2ms/step - loss: 0.2006 - acc: 0.9316
Epoch 52/60
50000/50000 [==============================] - 121s 2ms/step - loss: 0.1032 - acc: 0.9638
Epoch 53/60
50000/50000 [==============================] - 122s 2ms/step - loss: 0.1977 - acc: 0.9339
Epoch 54/60
50000/50000 [==============================] - 122s 2ms/step - loss: 0.0756 - acc: 0.9739
Epoch 55/60
50000/50000 [==============================] - 121s 2ms/step - loss: 0.0668 - acc: 0.9772
Epoch 56/60
50000/50000 [==============================] - 121s 2ms/step - loss: 0.0895 - acc: 0.9690
Epoch 57/60
50000/50000 [==============================] - 121s 2ms/step - loss: 0.1875 - acc: 0.9378
Epoch 58/60
50000/50000 [==============================] - 121s 2ms/step - loss: 0.1335 - acc: 0.9571
Epoch 59/60
50000/50000 [==============================] - 122s 2ms/step - loss: 0.1048 - acc: 0.9643
Epoch 60/60
50000/50000 [==============================] - 121s 2ms/step - loss: 0

In [31]:
preds = model.evaluate(x_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

10000/10000 [==============================] - 8s 834us/step
Loss = 8.14963706818
Test Accuracy = 0.2449


In [10]:
preds = model.evaluate(x_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

10000/10000 [==============================] - 10s 1ms/step
Loss = 2.01250006657
Test Accuracy = 0.6746


In [14]:
model.fit(x_train, y_train, epochs = 40, batch_size = 64,initial_epoch=35)

Epoch 36/40
50000/50000 [==============================] - 115s 2ms/step - loss: 0.0301 - acc: 0.9896
Epoch 37/40
50000/50000 [==============================] - 116s 2ms/step - loss: 0.0763 - acc: 0.9769
Epoch 38/40
50000/50000 [==============================] - 115s 2ms/step - loss: 0.0286 - acc: 0.9918
Epoch 39/40
50000/50000 [==============================] - 115s 2ms/step - loss: 0.0345 - acc: 0.9895
Epoch 40/40
50000/50000 [==============================] - 116s 2ms/step - loss: 0.0272 - acc: 0.9909


In [12]:
preds = model.evaluate(x_test, y_test, batch_size = 64)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

10000/10000 [==============================] - 4s 439us/step
Loss = 1.84914103546
Test Accuracy = 0.6978


In [15]:
preds = model.evaluate(x_test, y_test, batch_size = 64)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

10000/10000 [==============================] - 5s 462us/step
Loss = 2.01404643402
Test Accuracy = 0.7076


In [16]:
model.save('cifar10_trained_model.h5')